# Aerosol calibration

I will first try a relationship of the form

$ERF_{aci} = \sum_{i \in SLCF} \beta_i \log \left( 1 + \frac{E_i}{s_i} \right)$

where in each case $\beta_i$ is a scale factor and $s_i$ is a shape factor.

The problem with this is that it doesn't allow for interaction between terms, which we assume exist - shown in Thornhill et al. 2021 that the sum of aerosol ERFaci does not equal the sum of the components.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d

In [ ]:
ssp_df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
years = np.arange(1750,2101)
startyear = 1750
first_scenyear = 2015
last_scenyear = 2100
first_row = int(first_scenyear-startyear)
last_row = int(last_scenyear-startyear)

species = [  # in fair 1.6, order is important
    '|Sulfur',
    '|NOx',
    '|BC',
    '|OC',
    '|NH3',
    '|VOC'
]

# Assume that units coming out of aneris don't change. One day I'll do unit parsing
unit_convert = np.ones(40)
unit_convert[0] = 32/64
unit_convert[1] = 14/46

data_out = {}

scens = ['ssp245']
for scen in scens:
    data_out[scen] = np.ones((351, 7)) * np.nan
    data_out[scen][:,0] = years

    years_future = [2015] + list(range(2020,2101,10))
    for i, specie in enumerate(species):
        data_out[scen][:first_row,i+1] = ssp_df.loc[(ssp_df['Model']=='MESSAGE-GLOBIOM')&(ssp_df['Region']=='World')&(ssp_df['Scenario']==scen)&(ssp_df['Variable'].str.endswith(specie)),str(startyear):'2014']*unit_convert[i+1]
        f = interp1d(years_future, ssp_df.loc[(ssp_df['Model']=='MESSAGE-GLOBIOM')&(ssp_df['Region']=='World')&(ssp_df['Scenario']==scen)&(ssp_df['Variable'].str.endswith(specie)),'2015':'2100'].dropna(axis=1))
        data_out[scen][first_row:(last_row+1), i+1] = f(np.arange(first_scenyear, last_scenyear+1))*unit_convert[i+1]

In [ ]:
df = pd.DataFrame(data_out[scen], columns=['year','Sulfur','NOx','BC','OC','NH3','VOC'])

In [ ]:
df.set_index('year', inplace=True)
df

In [ ]:
def one_plus_log(x, x0, beta, s):
    erfaci = -np.sum(beta * np.log(1 + x/s), axis=1) - (-np.sum(beta * np.log(1 + x0/s)))
    return erfaci

In [ ]:
ts = one_plus_log(df, df.loc[1750], [190, 0.8, 0.5, 1.3, 0, 0], [50000, 43, 1000, 111, 1, 100])
pl.plot(ts)
ts[2005:2014].mean(), ts[2019]

In [ ]:
ts = one_plus_log(df, df.loc[1750], [2820000, 0, 0, 0, 0, 0], [1e8, 43, 1000, 111, 1, 100])
pl.plot(ts)
ts[2005:2014].mean(), ts[2019]

In [ ]:
ts = one_plus_log(df, df.loc[1750], [0, 0, 0, 1, 0, 0], [1e8, 43, 1000, 111, 1, 100])
pl.plot(ts)
ts[2005:2014].mean(), ts[2019]

In [ ]:
ts = one_plus_log(df, df.loc[1750], [0, 0, 5, 0, 0, 0], [1e8, 43, 1000, 111, 1, 100])
pl.plot(ts)
ts[2005:2014].mean(), ts[2019]

In [ ]:
ts = one_plus_log(df, df.loc[1750], [0, 0.012, 0, 0, 0, 0], [1e8, 43, 1000, 111, 1, 100])
pl.plot(ts)
ts[2005:2014].mean(), ts[2019]

In [ ]:
ts = one_plus_log(df, df.loc[1750], [0, 0, 0, 0, 0, -0.012], [1e8, 43, 1000, 111, 1, 100])
pl.plot(ts)
ts[2005:2014].mean(), ts[2019]

In [ ]:
ts = one_plus_log(df, df.loc[1750], [2820000, 0.012, 5, 1, 0, -0.012], [1e8, 43, 1000, 111, 1, 100])
pl.plot(ts)
ts[2005:2014].mean(), ts[2019]